In [1]:
import sys; sys.path.insert(0, '..') # So that we import the local copy of pyzx if you have installed from Github
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

from multiprocessing import Pool
import pyzx as zx
from pyzx import cnot_mapper, architecture # Note that this is the local pyzx package from this repository/fork, not the one installed through pip or other means.
from pyzx import circuit
from pyzx import *
from pyzx.parity_maps import CNOT_tracker, build_random_parity_map
from pyzx.linalg import Mat2
from pyzx.simplify import full_reduce

Load Circuits for CombRowCol

In [2]:
comb_circuit_folder = "../circuits/combs-ZX/"

n_qubits = [9]
n_cnots = [5,8,10,15,20]
n_non_cnots = [0,1,2,4,5,8,10,15,20]
loaded_circuits = {}
for qubits in n_qubits:
    for cnots in n_cnots:
        for non_cnots in n_non_cnots:
            circuit_properties = {"Qubits"    : qubits,
                                  "CNOTs"     : cnots,
                                  "Non-CNOTs" : non_cnots}
            path = os.path.join(comb_circuit_folder,
                                str(circuit_properties["Qubits"])+"qubits", 
                                str(circuit_properties["CNOTs"])+"cnots", 
                                str(circuit_properties["Non-CNOTs"])+"non-cnots")
            loaded_circuits[f"({qubits},{cnots},{non_cnots})"] = []
            if os.path.exists(path):
                for file_name in os.listdir(path):
                    loaded_circuits[f"({qubits},{cnots},{non_cnots})"].append(circuit.Circuit.from_qasm_file(os.path.join(path, file_name)))
            else:
                print("Directory Does Not Exist")

In [3]:
from IPython.display import display, Markdown

DEBUG = False
OUTER_DISPLAY = False
INNTER_DISPLAY = False

for qubits in n_qubits:
    for cnots in n_cnots:
        for non_cnots in n_non_cnots:
            arch = architecture.create_architecture(architecture.SQUARE, n_qubits=qubits)
            equal = 0
            ratios = []
            for INDEX in range(100):
                if False:
                    # Get circuit
                    circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]
                    # Draw circuit
                    OUTER_DISPLAY and display(zx.draw(circ))
                    # Decompose circuit into comb and holes
                    decomposition = CombDecomposition.from_circuit(circ.copy())
                    comb = decomposition.comb
                    # Draw comb
                    OUTER_DISPLAY and display(zx.draw(comb))
                    # Recompose circuit from comb and holes
                    new_circuit = CombDecomposition.to_circuit(decomposition)
                    # Display recomposed circuit
                    OUTER_DISPLAY and display(zx.draw(new_circuit))
                    # Check whether the recomposed circuit is the same as the original
                    equality = new_circuit.verify_equality(circ)
                    equal += equality
                else:
                    circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]
                    new_circ, Id = combrowcol(circ, arch, DEBUG, OUTER_DISPLAY, INNTER_DISPLAY)
                    gate_ratio = len(new_circ.gates)/len(circ.gates) - 1
                    ratios.append(gate_ratio)
                    equality = new_circ.verify_equality(circ)
                    equal += equality
                    #print(f"INDEX: {INDEX} , GATE RATIO : {gate_ratio}")
                    #print(f"INDEX: {INDEX} {equality}")
                    #equal += Id
                    #print(f"Reduced to identity : {INDEX} {Id}")
            print(f"Qubits : {qubits}, CNOTs : {cnots}, Non-CNOTs : {non_cnots} -> {equal}, Ratio : {sum(ratios)/len(ratios)}")

Qubits : 9, CNOTs : 5, Non-CNOTs : 0 -> 100, Ratio : 3.17
Qubits : 9, CNOTs : 5, Non-CNOTs : 1 -> 100, Ratio : 2.8033333333333332
Qubits : 9, CNOTs : 5, Non-CNOTs : 2 -> 100, Ratio : 2.402857142857143
Qubits : 9, CNOTs : 5, Non-CNOTs : 4 -> 100, Ratio : 1.768888888888889
Qubits : 9, CNOTs : 5, Non-CNOTs : 5 -> 100, Ratio : 1.8470000000000006
Qubits : 9, CNOTs : 5, Non-CNOTs : 8 -> 100, Ratio : 1.3784615384615382
Qubits : 9, CNOTs : 5, Non-CNOTs : 10 -> 100, Ratio : 1.2193333333333336
Qubits : 9, CNOTs : 5, Non-CNOTs : 15 -> 100, Ratio : 0.8970000000000004
Qubits : 9, CNOTs : 5, Non-CNOTs : 20 -> 100, Ratio : 0.8312000000000004
Qubits : 9, CNOTs : 8, Non-CNOTs : 0 -> 100, Ratio : 2.44875
Qubits : 9, CNOTs : 8, Non-CNOTs : 1 -> 100, Ratio : 2.311111111111112
Qubits : 9, CNOTs : 8, Non-CNOTs : 2 -> 100, Ratio : 2.202
Qubits : 9, CNOTs : 8, Non-CNOTs : 4 -> 100, Ratio : 1.9458333333333337
Qubits : 9, CNOTs : 8, Non-CNOTs : 5 -> 100, Ratio : 1.9307692307692315
Qubits : 9, CNOTs : 8, Non-CNO